In [1]:
from typing import Tuple
import re
import xml.etree.ElementTree as etree
from collections import defaultdict, OrderedDict
from functools import reduce

In [2]:
import numpy as np
import pandas as pd

## Read wikipedia dump

In [3]:
root = etree.parse('../data/kowiki.xml').getroot()

In [4]:
prefix = '{http://www.mediawiki.org/xml/export-0.10/}'
keywords = ['title', 'id', 'revision/text']

In [5]:
context = defaultdict(dict)

In [6]:
def parse_xml(node, key):
    return node.find(f'{prefix}{key}')

In [7]:
for page in root[1:]:
    data = {keyword: reduce(parse_xml, [page, *keyword.split('/')]).text for keyword in keywords}
    context[data['title']] = data

## Parser

In [8]:
regex_document = re.compile('\[\[(.[^:]+?)\]\]')
regex_table = re.compile('\{\{((?:.|\n)*)\}\}')
regex_table_column = re.compile('\|(.+?)\=([^\|]*)')
regex_bracket = re.compile('\((.+?)\)')
regex_redirect = re.compile('#넘겨주기 \[\[(.+?)\]\]')

regex_tags = OrderedDict({
    'horizontal_line': ('', re.compile('\-{4,9}')),
    'comment': ('', re.compile('##\s?(.*)')),
    'indent': (r'\1', re.compile('\:{1,}(.+?)$')),
    
    'header': (r'\1', re.compile('\={2,6}\#?(.+?)\#?\={2,6}')),
    'bold': (r'\1', re.compile("\'\'\'(.+?)\'\'\'")),
    'italic': (r'\1', re.compile("\'\'(.+?)\'\'")),
    'strike1': (r'\1', re.compile('~~(.+?)~~')),
    'strike2': (r'\1', re.compile('--(.+?)--')),
    'underline': (r'\1', re.compile('__(.+?)__')),
    'upper': (r'\1', re.compile('\^\^(.+?)\^\^')),
    'under': (r'\1', re.compile(',,(.+?),,')),
    
    'bigger': (r'\1', re.compile('\{\{\{\+[1-5] (.+?)\}\}\}')),
    'smaller': (r'\1', re.compile('\{\{\{\-[1-5] (.+?)\}\}\}')),
    'color': (r'\2', re.compile('\{\{\{\#(.+?) (.+?)\}\}\}')),
    'without_markup': (r'\1', re.compile('\{\{\{(.*)\}\}\}')),
    
    'macro_html': (r'\1', re.compile('\{\{\{\#\!html (.+?)\}\}\}')),
    'macro_wiki': (r'\2', re.compile('\{\{\{\#\!wiki (.+?)\n(.*)\}\}\}')),
    'macro_syntax': (r'\2', re.compile('\{\{\{\#\!syntax (.+?)\n(.*)\n\}\}\}', re.IGNORECASE)),
    'macro_color': (r'\2', re.compile('\{\{\{\#(.+?) (.+?)\}\}\}', re.IGNORECASE)),
    'macro_math': ('', re.compile('\[math\((.+?)\)\]', re.IGNORECASE)),
    'macro_date': ('', re.compile('\[date(time)?\]', re.IGNORECASE)),
    'macro_br': ('\n', re.compile('\<br \/\>', re.IGNORECASE)),
    'macro_include': ('', re.compile('\[include(.+?)\]', re.IGNORECASE)),
    'macro_index': ('', re.compile('\[목차\]')),
    'macro_index_': ('', re.compile('\[tableofcontents\]')),
    'macro_footnote': ('', re.compile('\[각주\]')),
    'macro_footnote_': ('', re.compile('\[footnote\]')),
    'macro_pagecount': ('', re.compile('\[pagecount(.+?)?\]', re.IGNORECASE)),
    'macro_age': ('', re.compile('\[age\(\)\]', re.IGNORECASE)),
    'macro_dday': ('', re.compile('\[dday\(\)\]', re.IGNORECASE)),
    'macro_tag': ('', re.compile('\<(.+?)\>')),

    'attach_': (r'\1', re.compile('\[\[파일:(.+?)\|(.+?)\]\]')),
    'attach': (r'\1', re.compile('\[\[파일:(.+?)\]\]')),
    
    'paragraph_': (r'\1', re.compile('\[\[#s-(.+?)\|(.+?)\]\]')),
    'paragraph': (r'\1', re.compile('\[\[#s-(.+?)\]]')),
    'link_paragraph_': (r'\1', re.compile('\[\[(.+?)#s-(.+?)\|(.+?)\]\]')),
    'link_paragraph': (r'\1', re.compile('\[\[(.+?)#s-(.+?)\]\]')),
    'link': (r'\1', re.compile('\[\[((?:(?!\|).)+?)\]\]')),
    'link_': (r'\1', re.compile('\[\[(.+?)\|(.+?)\]\]')),
    'link__': (r'\1', re.compile('\{\{(.+?)\|(.+?)\}\}')),
    
    'unordered_list': (r'\1', re.compile('\*{1,}:?(.*)')),
    'ordered_list': (r'\1', re.compile('\#{1,}:?(.*)')),
    
    
    'list_': (r'\1', re.compile('\|\*(.*)')),
    'list__': (r'\1', re.compile('\|[1Aa]\.\|(.*)')),
    'list___': (r'\1', re.compile('\|[1Aa]\.(.*)')),
    'unordred_list': (r'\1', re.compile('[ ]+\*(.*)')),
    'quote': (r'\1', re.compile('\>+\s?(.*)')),

    'footnote': ('', re.compile('\[\*[A-Za-z]? (.+?)\]')),    
})

In [9]:
def parse(text: str, verbose: bool = False) -> str:
    def _parse(text: str, target: str, tag: re.Pattern) \
        -> Tuple[str, int]:
        return tag.subn(target, text)
    
    if verbose:
        print(f'Parsing Regex {len(regex_tags.keys())} rules\n\t{text}')
    
    while True:
        count = 0
        for key, (target, tag) in regex_tags.items():
            text, count = _parse(text, target, tag)
            if count:
                if verbose:
                    print(f'Rule [{key}: {tag}]\n\t{text}')
                break
        if not count:
            break
            
    return text.strip()

In [10]:
def parse_table(text):
    for row in regex_table.findall(text):
        yield regex_table_column.findall(row)

## Get drama titles from lists

In [11]:
interested = ["대한민국의 텔레비전 드라마 목록/ㄱ", "대한민국의 텔레비전 드라마 목록/ㄴ", "대한민국의 텔레비전 드라마 목록/ㄷ", "대한민국의 텔레비전 드라마 목록/ㄹ", "대한민국의 텔레비전 드라마 목록/ㅁ", "대한민국의 텔레비전 드라마 목록/ㅂ", "대한민국의 텔레비전 드라마 목록/ㅅ", "대한민국의 텔레비전 드라마 목록/ㅇ", "대한민국의 텔레비전 드라마 목록/ㅈ", "대한민국의 텔레비전 드라마 목록/ㅊ", "대한민국의 텔레비전 드라마 목록/ㅋ", "대한민국의 텔레비전 드라마 목록/ㅌ", "대한민국의 텔레비전 드라마 목록/ㅍ", "대한민국의 텔레비전 드라마 목록/ㅎ", "대한민국의 텔레비전 드라마 목록/알파벳", "대한민국의 텔레비전 드라마 목록/숫자", ]

In [12]:
titles = set()

In [13]:
for inter in interested:
    document = context[inter]['revision/text']
    matches = regex_document.findall(document)
    
    for match in matches:
        name, *_ = match.split('|')
        name, *_ = name.split('#')
        if name != '대한민국의 텔레비전 드라마 목록':
            titles.add(name)

## Get metadata from document

In [14]:
columns = {
    '방송 기간': ['방송기간', '방송 기간'],
    '방송 시간': ['방송 시간'],
    '방송 횟수': ['횟수', '방송 횟수'],
    '장르': ['장르'],
    '채널': ['채널', '방송사', '방송 채널', '방송채널'],
#    '제작사': ['제작사', '제작자', '제작'],
#    '극본': ['극본', '대본', '각본'],
#    '출연자': ['출연자', '출연', '출연진'],
}

In [15]:
data = defaultdict(dict)
notexists = []
notparser = []

In [16]:
for title in titles:
    try:
        if '(드라마)' not in title and f'{title} (드라마)' in context:
            document, redirect = context[f'{title} (드라마)']['revision/text'], True
        else:
            document, redirect = context[title]['revision/text'], True
        while redirect:
            document, redirect = regex_redirect.subn(r'\1', document)
            if redirect:
                document = context[document]['revision/text']
    except KeyError:
        notexists.append(title)
    
    d = defaultdict(str)    
    for table in parse_table(parse(document)):
        for row in table:
            try:
                key, value = map(str.strip, filter(len, row))
            except ValueError:
                continue
            key = next((ckey for ckey, cvalues in columns.items() if any(cvalue in key for cvalue in cvalues)), False)
            if key:
                key = key.strip()
                d[key] = f'{d[key]} {value}'
            
    if not d:
        notparser.append(title)
    else:
        data[title] = d

In [17]:
print(f'{len(notparser)} pages are not parsable')
print(f'{len(notexists)} pages are not exists')

14 pages are not parsable
16 pages are not exists


## Create table from data

In [18]:
table_columns = list(columns.keys())
table = np.empty((0, len(table_columns) + 1))

for title, values in data.items():
    table = np.vstack((table, np.array([
        regex_bracket.sub('', title).replace(' ', '').strip(), *tuple(map(lambda c: values[c], table_columns))
    ])))

In [19]:
df = pd.DataFrame(table)
df.columns = ['제목', *table_columns]

### Parse datetime

In [20]:
regex_date = re.compile('(.+?)년(.*)월(.*)일')

In [21]:
date_start = []
for index, date in enumerate(df['방송 기간']):
    ds, *de = map(regex_date.findall, map(str.strip, date.split('~' if '~' in date else '-')))
    ds, *_ = ds or ['unknown']
    try:
        date_start.append(pd.datetime(*tuple(map(int, ds))))
    except (ValueError, TypeError):
        date_start.append('unknown')
    
assert len(date_start) == np.size(df, 0)

In [22]:
df['방송 시작'] = pd.Series(date_start)

## Show Dataframe

In [23]:
print(df.shape)
df.head()

(1759, 7)


,제목,방송 기간,방송 시간,방송 횟수,장르,채널,방송 시작
0,자꾸만보고싶네,2000년 9월 18일 ~ 2001년 3월 30일,월요일 ~ 금요일 밤 (시간) 8시 45분 ~ 9시 15분,128부작,드라마,SBS TV,2000-09-18 00:00:00
1,사랑하는은동아,2015년 5월 29일 ~ 2015년 7월 18일,"매주 금요일, 토요일 밤 (시간) 8시 40분 (1회 ~ 14회) 매주 금요일,...",16부작,텔레비전 드라마,JTBC,2015-05-29 00:00:00
2,진주목걸이,2003년 9월 20일 ~ 2004년 3월 14일,"토요일, 일요일 저녁 7시 50분 ~ 밤 8시 50분",52부작,드라마 드라마 음악,KBS 2TV,2003-09-20 00:00:00
3,거짓말,1998년 3월 30일 ~ 1998년 6월 2일,월~화 오후 9시 50분,20회,드라마,한국방송공사,1998-03-30 00:00:00
4,야인시대,2002년 7월 29일 ~ 2003년 9월 30일,9시 55분,124부작 + 스페셜,"시대극, 다큐드라마",SBS,2002-07-29 00:00:00


In [24]:
df.to_csv('../results/wikipedia.csv', index=None)